# Task 2: Visualize hidden represenations of a model

## Part 1: Extract and save representations

In [1]:
import h5py

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
!pip install datasets

# import dependencies
import torch

from datasets import load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names
from transformers import XGLMTokenizer, XGLMTokenizerFast, XGLMForCausalLM, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from torch.utils.data import DataLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00


In [3]:
MODEL_NAME = "facebook/xglm-564M"
DATASET_NAME = "facebook/flores"
BATCH_SIZE = 2
RANDOM_SUBSET_SIZE = 2

LANGUAGES = [
    "eng_Latn",
    "spa_Latn",
    "deu_Latn",
    "arb_Arab",
    "tam_Taml",
    "quy_Latn"
]

In [4]:
# Load dataset
flores_dataset={}
flores_dataset_builder={}
for language in LANGUAGES :
    dataset = load_dataset("facebook/flores", language)
    flores_dataset[language] = dataset

# Limit dataset to a random subset of 200 sentences for each language
random_subset = {lang: flores_dataset[lang]['dev'].shuffle(seed=42).select(range(RANDOM_SUBSET_SIZE)) for lang in LANGUAGES}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

...

Generating devtest split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [5]:
# tokenize the data

# load a pre-trained tokenizer from the huggingface hub
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# gpt2 does not have a padding token, so we have to add it manually
if MODEL_NAME == "gpt2":
    tokenizer.add_special_tokens({'pad_token': tokenizer.unk_token})

# specify the tokenization function
def tokenization(example):
    return tokenizer(example['sentence'], padding='max_length', truncation=True, max_length=42)

# tokenize your random subset
tokenized_datasets = {}
for language, dataset in random_subset.items():
    tokenized_datasets[language] = dataset.map(tokenization, batched=True)

tokenizer_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [6]:
for language, dataset in tokenized_datasets.items():
    dataset.set_format(type='torch')

data_loaders = {}
for language, dataset in tokenized_datasets.items():
    data_loaders[language] = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, output_hidden_states=True)

# Set model to evaluation mode
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [8]:
# TODO: your code goes here

# Function to extract hidden representations
def extract_hidden_representations(example, model):
    # Move input to CPU
    input_ids = example['input_ids']
    attention_mask = example['attention_mask']
    with torch.no_grad():
        # Move input to GPU if available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

    hidden_states = outputs.hidden_states  # Get hidden states from all layers
    return hidden_states

# Initialize HDF5 file for storage
with h5py.File("representations.h5", "w") as f:
    for lang, lang_dataset in data_loaders.items():
        print(f'Processing {lang}....')
        for example in lang_dataset:
            # Extract hidden representations
            hidden_states = extract_hidden_representations(example, model)
            # print(hidden_states.size())
            # Iterate through layers and tokens
            for layer, layer_hidden_states in enumerate(hidden_states):
                # print(layer_hidden_states.size())
                # print(example['attention_mask'][0].size())
                sentence_index = 0
                for sentence in layer_hidden_states:
                    for token_index, token_hidden_state in enumerate(sentence):
                        # Skip padding tokens
                        if (example['attention_mask'][sentence_index][token_index] == 0):
                            continue
                        # Save hidden representation for each token
                        # print(token_index)
                        f.create_dataset(
                            f"{lang}/Sample_{example['id'][sentence_index]}/layer_{layer}/token_{token_index}",
                            data=token_hidden_state.numpy()
                        )

                    # Calculate mean-pooled sentence representation
                    mean_pooled_sentence = np.mean(sentence.numpy(), axis=0)

                    # Save mean-pooled sentence representation
                    # print(f'Creating {lang}/Sample #{example["id"][sentence_index]}/mean_pooled_sentence')
                    f.create_dataset(
                        f"{lang}/Sample_{example['id'][sentence_index]}/layer_{layer}/mean_pooled_sentence",
                        data=mean_pooled_sentence
                    )
                    # print(f'Created {lang}/Sample #{example["id"][sentence_index]}/mean_pooled_sentence')
                    sentence_index += 1

        print('Done.')
        print()



Processing eng_Latn....
Done.

Processing spa_Latn....
Done.

Processing deu_Latn....
Done.

Processing arb_Arab....
Done.

Processing tam_Taml....
Done.

Processing quy_Latn....
Done.



## Part 2: Visualize representations

In [9]:
# Open the HDF5 file in read mode
with h5py.File("representations.h5", "r") as f:
    # Iterate through the groups in the file (each language)
    for lang in f.keys():
        print(f"Language: {lang}")
        # Iterate through the datasets (each example)
        for example_id in f[lang].keys():
            print(f"Example ID: {example_id}")
            # Print the keys of the datasets within the example
            print("Datasets:")
            for dataset_name in f[lang][example_id].keys():
                print(dataset_name)
                # Read and print the data from the dataset
                data = f[lang][example_id][dataset_name]
                print(data)

Language: arb_Arab
Example ID: Sample_969
Datasets:
layer_0
<HDF5 group "/arb_Arab/Sample_969/layer_0" (43 members)>
layer_1
<HDF5 group "/arb_Arab/Sample_969/layer_1" (43 members)>
layer_10
<HDF5 group "/arb_Arab/Sample_969/layer_10" (43 members)>
layer_11
<HDF5 group "/arb_Arab/Sample_969/layer_11" (43 members)>
layer_12
<HDF5 group "/arb_Arab/Sample_969/layer_12" (43 members)>
layer_13
<HDF5 group "/arb_Arab/Sample_969/layer_13" (43 members)>
layer_14
<HDF5 group "/arb_Arab/Sample_969/layer_14" (43 members)>
layer_15
<HDF5 group "/arb_Arab/Sample_969/layer_15" (43 members)>
layer_16
<HDF5 group "/arb_Arab/Sample_969/layer_16" (43 members)>
layer_17
<HDF5 group "/arb_Arab/Sample_969/layer_17" (43 members)>
layer_18
<HDF5 group "/arb_Arab/Sample_969/layer_18" (43 members)>
layer_19
<HDF5 group "/arb_Arab/Sample_969/layer_19" (43 members)>
layer_2
<HDF5 group "/arb_Arab/Sample_969/layer_2" (43 members)>
layer_20
<HDF5 group "/arb_Arab/Sample_969/layer_20" (43 members)>
layer_21
<HDF5 g